# 🧠 Predicción del Mundial 2022 con un Modelo de Poisson

En este notebook aplicamos un modelo probabilístico basado en la distribución de Poisson para predecir los resultados del Mundial de Fútbol de 2022. Se utilizan datos históricos de todos los mundiales desde 1930 para estimar la "fuerza" ofensiva y defensiva de cada selección, y se simula el torneo completo partido a partido.


## 🔁 Carga de datos

Importamos las librerías necesarias y cargamos los conjuntos de datos limpios, así como las tablas de grupos.


In [123]:
import pandas as pd
from scipy.stats import poisson
import pickle

In [124]:
dict_table = pickle.load(open('dict_table', 'rb'))
df_data_historica = pd.read_csv('clean_fifa_worldcup_historical_data.csv')
df_fixture = pd.read_csv('clean_fifa_worldcup_fixture.csv')

## 1. Cálculo de la fuerza de los equipos

Construimos un modelo de fuerza basado en los goles marcados y encajados por cada selección a lo largo de los mundiales anteriores. Para ello se calculan promedios de goles por partido, tanto a favor como en contra, considerando partidos como local y visitante.


In [125]:
#Creamos un dataframe para los equipos locales y otro para los visitantes
df_home = df_data_historica[['HomeTeam', 'HomeGoals', 'AwayGoals']]
df_away = df_data_historica[['AwayTeam', 'HomeGoals', 'AwayGoals']]

In [126]:
df_home = df_home.rename(columns = {'HomeTeam': 'Team', 'HomeGoals': 'GoalsScored', 'AwayGoals': 'GoalsConceded'})
df_away = df_away.rename(columns = {'AwayTeam': 'Team', 'HomeGoals': 'GoalsConceded', 'AwayGoals': 'GoalsScored'})

In [127]:
#Concatenamos ambos df, hacemos group por team y calculamos promedio de goles marcados y recibidos
df_team_strength = pd.concat([df_home, df_away], ignore_index = True).groupby('Team').mean()
df_team_strength

,GoalsScored,GoalsConceded
Team,,
Algeria,1.000000,1.461538
Angola,0.333333,0.666667
Argentina,1.717949,1.166667
Australia,0.812500,1.937500
Austria,1.576923,1.692308
...,...,...
Uruguay,1.603774,1.339623
Wales,0.800000,0.800000
West Germany,2.050847,1.254237


# 2. Modelo de predicción de partidos

Usamos la distribución de Poisson para estimar la probabilidad de cada posible resultado entre dos equipos. A partir de esas probabilidades, calculamos la expectativa de puntos (victoria = 3, empate = 1, derrota = 0) para cada equipo en un enfrentamiento.


In [128]:
def predict_points(home, away):
    if home in df_team_strength.index and away in df_team_strength.index:
        #Goles marcados * Goles recibidos por el contrario
        lambda_home = df_team_strength.at[home, 'GoalsScored'] * df_team_strength.at[away, 'GoalsConceded']
        lambda_away = df_team_strength.at[away, 'GoalsScored'] * df_team_strength.at[home, 'GoalsConceded']
        prob_home, prob_away, prob_draw = 0, 0, 0
        #Consideramos todos los posibles resultados hasta el 10-10
        for x in range(0, 11): #Goles del local
            for y in range(0, 11): #Goles del visitante
                p = poisson.pmf(x, lambda_home) * poisson.pmf(y, lambda_away)
                if x > y:
                    prob_home +=p
                elif x == y:
                    prob_draw +=p
                else:
                    prob_away +=p
        points_home = 3 * prob_home + prob_draw
        points_away = 3 * prob_away + prob_draw
        return(points_home, points_away) #La función devuelve los puntos que recibiría cada equipo (0, 1 o 3)
    else:
        return (0, 0)

## 2.1 Testear función

In [129]:
predict_points('Argentina', 'Mexico') #Argentina ganaría a Mexico

(np.float64(2.319527107953938), np.float64(0.5332248445655723))

La función devuelve (0,0) para partidos con equipos como Qatar, que nunca han participado en un mundial y no tenemos datos.
Con este modelo, los equipos que nunca antes han jugado un mundial no pasarán de fase de grupos.

In [130]:
predict_points('Qatar (H)', 'Ecuador')

(0, 0)

# 3. Simulación del torneo completo

Simulamos todas las fases del torneo usando el modelo anterior:
1. Fase de grupos
2. Octavos de final
3. Cuartos de final
4. Semifinales
5. Final

Los resultados se actualizan dinámicamente tras cada ronda.


In [ ]:
#Dividimos el fixture en grupos, octavos, cuartos...
df_fixture_group_48 = df_fixture[:48].copy()
df_fixture_knockout = df_fixture[48:56].copy()
df_fixture_quarter = df_fixture[56:60].copy()
df_fixture_semifinal = df_fixture[60:62].copy()
df_fixture_final = df_fixture[62:].copy()

## 3.1. Fase de Grupos

Distribuimos los partidos por grupo, simulamos cada enfrentamiento y actualizamos las tablas de posiciones. Se clasifican los dos mejores equipos de cada grupo.


In [132]:
#Simulamos los partidos y actualizamos las tablas
#Dividimos los partidos de cada grupo
for group in dict_table:
    dict_table[group]['Pts'] = dict_table[group]['Pts'].astype(float) #Para solucionar warning de la nueva version de pandas
    teams_in_group = dict_table[group]['Team'].values
    df_fixture_group_6 = df_fixture_group_48[df_fixture_group_48['home'].isin(teams_in_group)]
    #Simulamos los partidos de cada grupo
    for index, row in df_fixture_group_6.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        dict_table[group].loc[dict_table[group]['Team'] == home, 'Pts'] += points_home
        dict_table[group].loc[dict_table[group]['Team'] == away, 'Pts'] += points_away
        
    dict_table[group] = dict_table[group].sort_values('Pts', ascending=False).reset_index()
    dict_table[group] = dict_table[group][['Team', 'Pts']]
    dict_table[group] = dict_table[group].round(0)

In [133]:
#Tablas actualizadas
dict_table

{'Group A':           Team  Pts
 0  Netherlands  4.0
 1      Senegal  2.0
 2      Ecuador  2.0
 3    Qatar (H)  0.0,
 'Group B':             Team  Pts
 0        England  6.0
 1          Wales  5.0
 2  United States  4.0
 3           Iran  2.0,
 'Group C':            Team  Pts
 0     Argentina  7.0
 1        Poland  6.0
 2        Mexico  4.0
 3  Saudi Arabia  1.0,
 'Group D':         Team  Pts
 0     France  7.0
 1    Denmark  6.0
 2    Tunisia  3.0
 3  Australia  2.0,
 'Group E':          Team  Pts
 0     Germany  7.0
 1       Spain  5.0
 2       Japan  3.0
 3  Costa Rica  2.0,
 'Group F':       Team  Pts
 0  Croatia  7.0
 1  Belgium  5.0
 2  Morocco  4.0
 3   Canada  0.0,
 'Group G':           Team  Pts
 0       Brazil  8.0
 1  Switzerland  4.0
 2       Serbia  3.0
 3     Cameroon  2.0,
 'Group H':           Team  Pts
 0     Portugal  6.0
 1      Uruguay  5.0
 2        Ghana  4.0
 3  South Korea  2.0}

## 3.2. Octavos de Final

Se actualiza el fixture con los clasificados de grupos y se simulan los enfrentamientos uno a uno.


In [ ]:
for group in dict_table:
    group_winner = dict_table[group].loc[0, 'Team']
    group_winner2 = dict_table[group].loc[1, 'Team']
    df_fixture_knockout.replace({f'Winners {group}': group_winner, f'Runners-up {group}': group_winner2}, inplace = True)
    
df_fixture_knockout['winner'] = '?'

In [136]:
#Utilizamos la función predict_points() para definir otra función que obtenga el ganador
def get_winner(df_fixture_updated):
    for index, row in df_fixture_updated.iterrows():
        home, away = row['home'], row['away']
        points_home, points_away = predict_points(home, away)
        if points_home > points_away:
            winner = home
        else:
            winner = away
        df_fixture_updated.loc[index, 'winner']  = winner 
    return(df_fixture_updated) 

In [137]:
get_winner(df_fixture_knockout)

,home,score,away,year,winner
48,Netherlands,Match 49,Wales,2022,Netherlands
49,Argentina,Match 50,Denmark,2022,Argentina
50,France,Match 52,Poland,2022,France
51,England,Match 51,Senegal,2022,England
52,Germany,Match 53,Belgium,2022,Germany
53,Brazil,Match 54,Uruguay,2022,Brazil
54,Croatia,Match 55,Spain,2022,Croatia
55,Portugal,Match 56,Switzerland,2022,Portugal


## 3.3 Cuartos de final

In [ ]:
#Función para actualizar fixture, nos servirá para cuartos, semifinal y final:
def update_tables(df_fixture_round1, df_fixture_round2):
    for index, row in df_fixture_round1.iterrows():
        winner = df_fixture_round1.loc[index, 'winner']
        match = df_fixture_round1.loc[index, 'score']
        df_fixture_round2.replace({f'Winners {match}': winner}, inplace = True)
        df_fixture_round2['winner'] = '?'
    return(df_fixture_round2)    

In [ ]:
#Actualizamos el fixture de cuartos
update_tables(df_fixture_knockout, df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,?
57,Netherlands,Match 57,Argentina,2022,?
58,Croatia,Match 60,Portugal,2022,?
59,England,Match 59,France,2022,?


In [142]:
#Obtenemos los ganadores
get_winner(df_fixture_quarter)

,home,score,away,year,winner
56,Germany,Match 58,Brazil,2022,Brazil
57,Netherlands,Match 57,Argentina,2022,Netherlands
58,Croatia,Match 60,Portugal,2022,Portugal
59,England,Match 59,France,2022,France


## 3.4 Semifinal

In [143]:
#Actualizamos fixture
update_tables(df_fixture_quarter, df_fixture_semifinal)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,?
61,France,Match 62,Portugal,2022,?


In [144]:
#Obtenemos los ganadores
get_winner(df_fixture_semifinal)

,home,score,away,year,winner
60,Netherlands,Match 61,Brazil,2022,Brazil
61,France,Match 62,Portugal,2022,France


## 3.5 Final

In [145]:
#Actualizamos fixture
update_tables(df_fixture_semifinal, df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,?
63,Brazil,Match 64,France,2022,?


In [146]:
#Obtenemos el ganador
get_winner(df_fixture_final)

,home,score,away,year,winner
62,Losers Match 61,Match 63,Losers Match 62,2022,Losers Match 62
63,Brazil,Match 64,France,2022,Brazil


## ✅ Resultados finales del torneo

Una vez simuladas todas las fases, obtenemos un ganador del torneo. Este modelo es determinista en cada ejecución (salvo que incorpores simulaciones múltiples o aleatoriedad), por lo que los resultados son reproducibles.

> 🧪 Posibles mejoras: el modelo podría refinarse incorporando datos recientes de clasificación, ranking FIFA, partidos amistosos, o aplicando un enfoque bayesiano más sofisticado.
